In [ ]:
!pip install --upgrade xlrd

     |████████████████████████████████| 96 kB 2.9 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
import pandas as pd
import numpy as np
import math
from scipy.integrate import quad

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_excel(r'/content/drive/MyDrive/Data_BlackKoffer_intern/DATA1/53.xls',sheet_name='results',header=None)

In [ ]:
df

In [ ]:
Vt = 0

In [ ]:
def f1(sigma,t):   # when we use this function, t will go from 1 till 300 (not 0-299)
  result = 1
  for k in range(20*math.floor((t-1)/20)+1, t+1):  # we want it to go from start ... t, that's why used t+1                  
    i = k-1  # id of data point, since k is id+1 (k is in column no.0 which is id+1)
    val = math.exp(-df[1][i]*df[1][i] / (2*sigma*sigma))
    val = val / (math.sqrt(2*math.pi)*sigma)
    result = result * val
  return result

In [ ]:
def f2(t):
  result = 1
  for k in range(20*math.floor((t-1)/20)+1, t+1):                                   # 
    i = k-1
    val = math.pi*(1 + df[1][i]*df[1][i])
    val = 1 / val
    result = result * val
  return result

In [ ]:
def m1(t):
  val = lambda t : quad(f1, 0.1,2, args=(t,))
  ans,err = val(t)
  return (1/1.9)*ans

In [ ]:
def m2(t):
  return f2(t)

In [ ]:
Pm1 = 0.5
Pm2 = 0.5

In [ ]:
def Pm1Xt(t):
  val = Pm1 * m1(t)
  val = val / (Pm1*m1(t) + Pm2*m2(t))
  return val

In [ ]:
def Pm2Xt(t):
  val = Pm2 * m2(t)
  val = val / (Pm1*m1(t) + Pm2*m2(t))
  return val

We win 2dollars on winning bet and loss $40 on lossing bet

In [ ]:
def sigmaapprox(t):
  start = 20*math.floor((t-1)/20)+1                                                # 
  end = t
  if start==t: 
    return 0  # no variance if there is only one data point
  result = 1 / (end-start+1)
  val = 0
  for k in range(start,end+1):
    i = k-1
    val += df[1][i]*df[1][i]
  result = result * val
  result = math.sqrt(result)
  
  return result

In [ ]:
def pg(t):
  sigma = sigmaapprox(t)
  if sigma==0:
    return 0
  def integrand(x):
      result = 1 / (sigma*math.sqrt(2*math.pi))
      result = result * math.exp(-x*x / (2*sigma*sigma))
      return result
  phi,err = quad(integrand,-float('inf'),4)
  result = 1 - 2*(1-phi)
  
  return result    

In [ ]:
def pc():
  return 0.844

In [ ]:
def compute_expValue(t):
  if t%20==0 or t%20==1: return 0   # t is the first data point of a trial set
  Pg = pg(t)
  Pc = pc()
  V = Pm1Xt(t-1) * (2*Pg - (1-Pg)*40) + Pm2Xt(t-1) * (2*Pc - (1-Pc)*40)
  
  return V # exp value

We bet on trial t+1 if and only if Vt > 0 

In [ ]:

def predict(start,end,beta):
  total = 0
  plist = []  # the list containing the bet values (1/0)
  probList = []
  for i in range(start,end):
    # i is the time stamp
    Vt = compute_expValue(i)
    prob = math.exp(beta*Vt) / (1 + math.exp(beta*Vt))
    probList.append(prob)
    if prob>0.5:
      plist.append(1)
      if df[1][i]<=4 and df[1][i]>=-4: 
        total+=2
      else: 
        total-=40
    else: plist.append(0)        
  return (plist,probList,total)       

Now we train the model (the parameter beta)

In [ ]:
Y = df[4]

In [ ]:
def cost_func(prediction,Y):
  total = 0
  for i in range(1,160):  # i is timestamp
    total -= (Y[i]*math.log(prediction[i]) + (1-Y[i])*math.log(1-prediction[i]))

  total /= 160
  return total

In [ ]:
def gradient_cost_func(beta):
  total = 0
  plist,probList,total = predict(0,160,beta)
  for i in range(3,160):
    total += (probList[i]-Y[i])*compute_expValue(i-1)
  
  total = total/160 
  return total

In [ ]:
beta = 1
alpha = 0.05
i = 0
while True:
  if i==100: break
  
  if i==0:
    pre = beta
    i+=1
  else:
    beta = beta - alpha*gradient_cost_func(beta)
    print("beta: ",beta)
    if abs(beta-pre)<1e-3: break
    else: pre = beta
    i+=1

Now we get the results

In [ ]:
plist,probList,total = predict(160,300,beta)

print(len(plist))
print(total)

In [ ]:
print(plist)

In [ ]:
Y = df[4]
print(Y[0])

In [ ]:
total = 0
for i in range(0,140):
  if (plist[i]==0 and Y[i+160]==0) or (plist[i]==1 and Y[i+160]==1): total+=1
print("The accuracy score of the base model is: ",total/140*100,"%")  

The accuracy score of the base model is:  79.28571428571428 %
